## Minnesota State COVID Response Analysis
This notebook contains the work to identify associations between the Minnesota state governmental response and the COVID-19 case count throughout the pandemic.


## Data Cleanup
As with most data mining projects, we will need to clean up the given data file in order to focus on the goal at hand. The "all-states-history.csv" file is a dataset of U.S. COVID-19 cases and deaths dating from the start of the pandemic to 11/29/20 and was sourced from [The Covid Tracking Project](https://covidtracking.com/data). We are analyzing 3 periods throughout this timeline:

- Early Breakout (Early March -> May)
- Summer (June -> August)
- Fall/Present (September -> Late November)

We will divide up the data into 3 different frames according to these periods.

In order to analyze with state policy actions, we will merge data from the [Oxford Covid-19 Government Response Tracker](https://github.com/OxCGRT/covid-policy-tracker) github dataset titled 'state-policies.csv'. 

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import squarify
import seaborn as sns

Initializing the dataframes

In [2]:
# COVID tracking project data
covid_data = pd.read_csv('all-states-history.csv')

# state plicy data
policy_data = pd.read_csv('state-policies.csv')

/Applications/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Cleaning up Covid data to only include Minnesota instances and the appropriate attributes

In [3]:
#isolating the columns we need
columns_to_show = ['date','state','death','deathConfirmed','deathIncrease','hospitalized','hospitalizedIncrease','negative'
                   ,'negativeIncrease','positive','positiveIncrease','totalTestResults','totalTestResultsIncrease']

#isolating only for MN data and putting in order March->November
covid_clean_data = covid_data[covid_data['state'] == 'MN']
covid_clean_data = covid_clean_data[columns_to_show]
covid_clean_data = covid_clean_data.iloc[::-1]

#reindexing for weekly processing 
covid_clean_data['date'] = covid_clean_data['date'].astype('datetime64[ns]')
covid_clean_data = covid_clean_data.set_index('date')

# isolating the columns that need to be summed when converting to weekly index
columns_to_sum = covid_clean_data[['deathIncrease','hospitalizedIncrease','negativeIncrease','positiveIncrease','totalTestResultsIncrease']]
weekly_data = columns_to_sum.resample('W', label='right', closed='right').sum()
weekly_data = weekly_data.reset_index()

# converting remaining non-sum columns to weekly index
remaining_cols = covid_clean_data[['state','death','deathConfirmed','hospitalized', 'negative','positive','totalTestResults']]
remaining_cols = remaining_cols.resample('W').backfill().reset_index()
remaining_cols.head(39)

#merging and resetting the datframe order to be more clear
covid_clean_data = pd.merge(remaining_cols, weekly_data, on='date').fillna(0)
covid_clean_data = covid_clean_data[['date','state','death','deathIncrease','deathConfirmed','hospitalized', 'hospitalizedIncrease','negative',
                        'negativeIncrease','positive', 'positiveIncrease','totalTestResults','totalTestResultsIncrease']]

covid_clean_data.head(39)

,date,state,death,deathIncrease,deathConfirmed,hospitalized,hospitalizedIncrease,negative,negativeIncrease,positive,positiveIncrease,totalTestResults,totalTestResultsIncrease
0,2020-03-08,MN,0.0,0,0.0,0.0,0,48.0,12,2.0,1,50.0,13
1,2020-03-15,MN,0.0,0,0.0,0.0,0,1387.0,1339,128.0,126,1515.0,1465
2,2020-03-22,MN,1.0,1,0.0,12.0,12,4511.0,3124,349.0,221,4860.0,3345
3,2020-03-29,MN,9.0,8,0.0,75.0,63,18566.0,14055,725.0,376,19291.0,14431
4,2020-04-05,MN,29.0,20,0.0,202.0,127,28090.0,9524,1217.0,492,29307.0,10016
5,2020-04-12,MN,70.0,41,0.0,361.0,159,38030.0,9940,1864.0,647,39894.0,10587
6,2020-04-19,MN,134.0,64,0.0,574.0,213,46288.0,8258,2914.0,1050,49202.0,9308
7,2020-04-26,MN,272.0,138,0.0,829.0,255,59161.0,12873,5512.0,2598,64673.0,15471
8,2020-05-03,MN,419.0,147,419.0,1199.0,370,83433.0,24272,9411.0,3899,92844.0,28171
9,2020-05-10,MN,578.0,159,578.0,1657.0,458,111109.0,27676,13552.0,4141,124661.0,31817


Cleaning up state policy dataframe:

In [4]:
#isolating data only about the current state of interest, Minnesota
policy_clean_data = policy_data[policy_data['RegionName'] == 'Minnesota']
#deleting rows whose dates are outside of the scope of this project
policy_clean_data = policy_clean_data.iloc[60:] #delete the first 60 rows due to their January - February dates
policy_clean_data = policy_clean_data.iloc[:-3,] #as well as the last 3 rows due to their December dates

#declaring and extracting columns of interest from the original dataset
columns_of_interest = ['RegionName', 'Jurisdiction', 'Date', 'C1_School closing', 'C2_Workplace closing', 
                       'C3_Cancel public events', 'C4_Restrictions on gatherings', 'C6_Stay at home requirements', 
                       'C7_Restrictions on internal movement', 'C8_International travel controls', 
                       'H1_Public information campaigns', 'H2_Testing policy', 'H3_Contact tracing', 
                       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines', 
                       'H6_Facial Coverings', 'M1_Wildcard']
policy_clean_data = policy_clean_data[columns_of_interest].fillna(0)

# reformating date section
policy_clean_data = policy_clean_data.reset_index(drop = True)
from datetime import datetime as dt

for i in range(policy_clean_data.shape[0]):
    date_string = str(policy_clean_data['Date'][i])
    policy_clean_data['Date'][i] = dt.strptime(date_string, "%Y%m%d")


policy_clean_data = policy_clean_data.set_index('Date')

policy_clean_data = policy_clean_data.resample('W').backfill().reset_index()

policy_clean_data = policy_clean_data.loc[1:39].reset_index(drop=True).drop(['RegionName','Jurisdiction'],axis=1)

policy_clean_data = policy_clean_data.rename(columns = {"Date":"date"})
policy_clean_data.head(41)

<ipython-input-4-e97690649880>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  policy_clean_data['Date'][i] = dt.strptime(date_string, "%Y%m%d")


,date,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,H6_Facial Coverings,M1_Wildcard
0,2020-03-08,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
1,2020-03-15,0.0,0.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
2,2020-03-22,3.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
3,2020-03-29,3.0,3.0,2.0,4.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
4,2020-04-05,3.0,3.0,2.0,4.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
5,2020-04-12,3.0,3.0,2.0,4.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
6,2020-04-19,3.0,3.0,2.0,4.0,2.0,1.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0
7,2020-04-26,3.0,3.0,2.0,4.0,2.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0
8,2020-05-03,3.0,3.0,2.0,4.0,2.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,1.0,0.0
9,2020-05-10,3.0,2.0,2.0,4.0,2.0,1.0,1.0,2.0,2.0,1.0,0.0,0.0,1.0,0.0


Changing values of policy_clean_data to the type of policy each number corresponds to:

In [5]:
#C1_School closing
policy_clean_data['C1_School closing'] = policy_clean_data['C1_School closing'].replace(0, 'no measures')
policy_clean_data['C1_School closing'] = policy_clean_data['C1_School closing'].replace(1, 'recommend closing or all schools open with alterations resulting in significant differences compared to non-Covid-19 operations')
policy_clean_data['C1_School closing'] = policy_clean_data['C1_School closing'].replace(2, 'require closing (only some levels or categories, eg just high school, or just public schools)')
policy_clean_data['C1_School closing'] = policy_clean_data['C1_School closing'].replace(3, 'require closing all levels')
#C2_Workplace closing
policy_clean_data['C2_Workplace closing'] = policy_clean_data['C2_Workplace closing'].replace(0, 'no measures')
policy_clean_data['C2_Workplace closing'] = policy_clean_data['C2_Workplace closing'].replace(1, 'recommend closing (or recommend work from home)')
policy_clean_data['C2_Workplace closing'] = policy_clean_data['C2_Workplace closing'].replace(2, 'require closing (or work from home) for some sectors or categories of workers')
policy_clean_data['C2_Workplace closing'] = policy_clean_data['C2_Workplace closing'].replace(3, 'require closing (or work from home) for all-but-essential workplaces (eg grocery stores, doctors)')
#C3_Cancel public events
policy_clean_data['C3_Cancel public events'] = policy_clean_data['C3_Cancel public events'].replace(0, 'no measures')
policy_clean_data['C3_Cancel public events'] = policy_clean_data['C3_Cancel public events'].replace(1, 'recommend cancelling')
policy_clean_data['C3_Cancel public events'] = policy_clean_data['C3_Cancel public events'].replace(2, 'require cancelling')
#C4_Restrictions on gatherings                                                                                            
policy_clean_data['C4_Restrictions on gatherings'] = policy_clean_data['C4_Restrictions on gatherings'].replace(0, 'no restrictions')                                                                                              
policy_clean_data['C4_Restrictions on gatherings'] = policy_clean_data['C4_Restrictions on gatherings'].replace(1, 'restrictions on very large gatherings (the limit is above 1000 people)')                                                                                              
policy_clean_data['C4_Restrictions on gatherings'] = policy_clean_data['C4_Restrictions on gatherings'].replace(2, 'restrictions on gatherings between 101-1000 people')                                                                                              
policy_clean_data['C4_Restrictions on gatherings'] = policy_clean_data['C4_Restrictions on gatherings'].replace(3, 'restrictions on gatherings between 11-100 people')                                                                                              
policy_clean_data['C4_Restrictions on gatherings'] = policy_clean_data['C4_Restrictions on gatherings'].replace(4, 'restrictions on gatherings of 10 people or less')
#C6_Stay at home requirements                                                                                            
policy_clean_data['C6_Stay at home requirements'] = policy_clean_data['C6_Stay at home requirements'].replace(0, 'no measures')
policy_clean_data['C6_Stay at home requirements'] = policy_clean_data['C6_Stay at home requirements'].replace(1, 'recommend not leaving house')
policy_clean_data['C6_Stay at home requirements'] = policy_clean_data['C6_Stay at home requirements'].replace(2, 'require not leaving house with exceptions for daily exercise, grocery shopping, and "essential" trips')
policy_clean_data['C6_Stay at home requirements'] = policy_clean_data['C6_Stay at home requirements'].replace(3, 'require not leaving house with minimal exceptions (eg allowed to leave once a week, or only one person can leave at a time, etc)')
#C7_Restrictions on internal movement
policy_clean_data['C7_Restrictions on internal movement'] = policy_clean_data['C7_Restrictions on internal movement'].replace(0, 'no measures')
policy_clean_data['C7_Restrictions on internal movement'] = policy_clean_data['C7_Restrictions on internal movement'].replace(1, 'recommend not to travel between regions/cities')
policy_clean_data['C7_Restrictions on internal movement'] = policy_clean_data['C7_Restrictions on internal movement'].replace(2, 'internal movement restrictions in place')
#C8_International travel controls
policy_clean_data['C8_International travel controls'] = policy_clean_data['C8_International travel controls'].replace(0, 'no restrictions')
policy_clean_data['C8_International travel controls'] = policy_clean_data['C8_International travel controls'].replace(1, 'screening arrivals')
policy_clean_data['C8_International travel controls'] = policy_clean_data['C8_International travel controls'].replace(2, 'quarantine arrivals from some or all regions')
policy_clean_data['C8_International travel controls'] = policy_clean_data['C8_International travel controls'].replace(3, 'ban arrivals from some regions')
policy_clean_data['C8_International travel controls'] = policy_clean_data['C8_International travel controls'].replace(4, 'ban on all regions or total border closure')
#H1_Public information campaigns
policy_clean_data['H1_Public information campaigns'] = policy_clean_data['H1_Public information campaigns'].replace(0, 'no Covid-19 public information campaign')
policy_clean_data['H1_Public information campaigns'] = policy_clean_data['H1_Public information campaigns'].replace(1, 'public officials urging caution about Covid-19')
policy_clean_data['H1_Public information campaigns'] = policy_clean_data['H1_Public information campaigns'].replace(2, 'coordinated public information campaign (eg across traditional and social media)')
#H2_Testing policy
policy_clean_data['H2_Testing policy'] = policy_clean_data['H2_Testing policy'].replace(0, 'no testing policy')
policy_clean_data['H2_Testing policy'] = policy_clean_data['H2_Testing policy'].replace(1, 'only those who both (a) have symptoms AND (b) meet specific criteria (eg key workers, admitted to hospital, came into contact with a known case, returned from overseas)')
policy_clean_data['H2_Testing policy'] = policy_clean_data['H2_Testing policy'].replace(2, 'testing of anyone showing Covid-19 symptoms')
policy_clean_data['H2_Testing policy'] = policy_clean_data['H2_Testing policy'].replace(3, 'open public testing (eg "drive through" testing available to asymptomatic people)')
#H3_Contact tracing
policy_clean_data['H3_Contact tracing'] = policy_clean_data['H3_Contact tracing'].replace(0, 'no contact tracing')
policy_clean_data['H3_Contact tracing'] = policy_clean_data['H3_Contact tracing'].replace(1, 'limited contact tracing; not done for all cases')
policy_clean_data['H3_Contact tracing'] = policy_clean_data['H3_Contact tracing'].replace(2, 'comprehensive contact tracing; done for all identified cases')
#H4_Emergency investment in healthcare
### not a code, simply records the monetary value in USD
policy_clean_data['H4_Emergency investment in healthcare'] = policy_clean_data['H4_Emergency investment in healthcare'].replace(0, 'no new spending that day')
#H5_Investment in vaccines
### same as H4
policy_clean_data['H5_Investment in vaccines'] = policy_clean_data['H5_Investment in vaccines'].replace(0, 'no new spending that day')                                                                               
#H6_Facial Coverings
policy_clean_data['H6_Facial Coverings'] = policy_clean_data['H6_Facial Coverings'].replace(0, 'no policy')
policy_clean_data['H6_Facial Coverings'] = policy_clean_data['H6_Facial Coverings'].replace(1, 'recommended')
policy_clean_data['H6_Facial Coverings'] = policy_clean_data['H6_Facial Coverings'].replace(2, 'required in some specified shared/public spaces outside the home with other people present, or some situations when social distancing not possible')
policy_clean_data['H6_Facial Coverings'] = policy_clean_data['H6_Facial Coverings'].replace(3, 'required in all shared/public spaces outside the home with other people present or all situations when social distancing not possible')
policy_clean_data['H6_Facial Coverings'] = policy_clean_data['H6_Facial Coverings'].replace(4, 'required outside the home at all times regardless of location or presence of other people')
#M1_Wildcard
policy_clean_data['M1_Wildcard'] = policy_clean_data['M1_Wildcard'].replace(0.0, 'none')
                                                                                              

Merging dataframes:

In [6]:
clean_data = pd.merge(covid_clean_data, policy_clean_data, on='date')

In [7]:
clean_data.head()

,date,state,death,deathIncrease,deathConfirmed,hospitalized,hospitalizedIncrease,negative,negativeIncrease,positive,...,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,H6_Facial Coverings,M1_Wildcard
0,2020-03-08,MN,0.0,0,0.0,0.0,0,48.0,12,2.0,...,no measures,no measures,screening arrivals,coordinated public information campaign (eg ac...,only those who both (a) have symptoms AND (b) ...,limited contact tracing; not done for all cases,no new spending that day,no new spending that day,no policy,none
1,2020-03-15,MN,0.0,0,0.0,0.0,0,1387.0,1339,128.0,...,recommend not leaving house,recommend not to travel between regions/cities,screening arrivals,coordinated public information campaign (eg ac...,only those who both (a) have symptoms AND (b) ...,limited contact tracing; not done for all cases,no new spending that day,no new spending that day,no policy,none
2,2020-03-22,MN,1.0,1,0.0,12.0,12,4511.0,3124,349.0,...,recommend not leaving house,recommend not to travel between regions/cities,screening arrivals,coordinated public information campaign (eg ac...,only those who both (a) have symptoms AND (b) ...,limited contact tracing; not done for all cases,no new spending that day,no new spending that day,no policy,none
3,2020-03-29,MN,9.0,8,0.0,75.0,63,18566.0,14055,725.0,...,require not leaving house with exceptions for ...,internal movement restrictions in place,screening arrivals,coordinated public information campaign (eg ac...,only those who both (a) have symptoms AND (b) ...,limited contact tracing; not done for all cases,no new spending that day,no new spending that day,no policy,none
4,2020-04-05,MN,29.0,20,0.0,202.0,127,28090.0,9524,1217.0,...,require not leaving house with exceptions for ...,internal movement restrictions in place,screening arrivals,coordinated public information campaign (eg ac...,only those who both (a) have symptoms AND (b) ...,limited contact tracing; not done for all cases,no new spending that day,no new spending that day,no policy,none


In [8]:
## Breaking down clean data into each period (earliest days at bottom of dataset)

early_breakout_data = clean_data[0:13]

summer_data = clean_data[13:26]

fall_data = clean_data[26:]

early_breakout_data.head(13)

bins = pd.cut(early_breakout_data['positiveIncrease'],4)

print(bins.shape)


(13,)


## Analysis

Important MN Stats:

- Population (mn.gov estimate): 5,680,337
- Land Area (estimate): 79,610.08 sq. mi.
- Population Density: 71.35 people/sq. mi.

Since we are performing a market basket analysis using the Apriori algorithm, we will need to discretize the data. To do so, we've implemented a function 'discretize_data':

In [9]:
# arr is the dataframe 
# k is the number of equal frequency bins
def discretize_data(arr, k):
    out = pd.DataFrame({'date': arr['date']})
    out['state'] = arr['state']
    cols = arr.columns[2:]
    for i in cols:
        bins = pd.cut(arr[i], k, 'retbins' == True, labels = list(range(k)))
        bin_range = pd.cut(arr[i],k)
        for j in range(k):
            count = 0
            for row in arr.index:
                if bins.loc[row] == j:
                    out.loc[row, i + " bin " +  str(bin_range.loc[count])] = 1
                count += 1
    out = out.fillna(0)
    return out      

Early Breakout Analysis:

In [ ]:
early_break_disc = discretize_data(early_breakout_data,4)
early_break_disc.head(10)